In [28]:
import Pkg; Pkg.activate("$(homedir())/.JuliaEnvironments/myanalysis");

using CairoMakie
using ColorSchemes
using Colors
using NCDatasets
using DataStructures: OrderedDict

using Printf
using JLD2

using Statistics
using GeoMakie
using NaturalEarth

#Pkg.activate("/Users/jablas001/Models/Julia/packages/IceAnalysis");
#using IceAnalysis
#Pkg.develop(PackageSpec(path="/Users/jablas001/Models/Julia/packages/IceAnalysis"));
#include("/Users/jablas001/Models/Julia/packages/IceAnalysis/src/IcePlots.jl")

  Activating project at `~/.JuliaEnvironments/myanalysis`


In [65]:
# For my profiles i need the 2D fields.
# Load dataset
ds_path    = "/Users/jablas001/Documents/Work/AWI/CalvingMIP/submission_new/"
exp        = "exp5_dt1"
exp_name   = "yelmo2D.nc"

# Define nc datasets of interest
clvmip_exp = NCDataset(joinpath(ds_path,exp,exp_name));

### Extract general fields too ###
# Topography
X      = clvmip_exp["xc"];
Y      = clvmip_exp["yc"];
Time   = clvmip_exp["time"]; #Time100
H_frnt = clvmip_exp["H_frnt"];
H_ice  = clvmip_exp["lithk"];
ux     = clvmip_exp["xvelmean"];
vy     = clvmip_exp["yvelmean"];
ux_ac  = clvmip_exp["xvelmean_ac"];
vy_ac  = clvmip_exp["yvelmean_ac"];
mask   = clvmip_exp["mask"];
dx     = 5000.0;
tsize  = size(Time)[1];
xsize  = size(X)[1];
profile_size = Int((xsize-1)*0.5 + 1);

In [66]:
# Create profile dimension
# === Caprona ===
sizeC = Int((450000-0)/dx) + 1;
xCA   = zeros(sizeC); # (-390 km, 0 km) and (-590 km, 450 km)
yCA   = zeros(sizeC);
xCB   = zeros(sizeC); # ( 390 km, 0 km) and ( 590 km, 450 km)
yCB   = zeros(sizeC);
xCC   = zeros(sizeC); # (-390 km, 0 km) and (-590 km, -450 km)
yCC   = zeros(sizeC);
xCD   = zeros(sizeC); # ( 390 km, 0 km) and ( 590 km, -450 km)
yCD   = zeros(sizeC);

# === Halbrane ===
sizeH = Int((740000-0)/dx) + 1;
xHA   = zeros(sizeH); #(-150 km, 0 km) and (-150 km, 740 km)
yHA   = zeros(sizeH);
xHB   = zeros(sizeH); # (150 km, 0 km) and ( 150 km, 740 km)
yHB   = zeros(sizeH);
xHC   = zeros(sizeH); # (-150 km, 0 km) and (-150 km, -740 km)
yHC   = zeros(sizeH);
xHD   = zeros(sizeH); # ( 150 km, 0 km) and ( 150 km, -740 km)
yHD   = zeros(sizeH);

# === Caprona ===
deltaXC = Int(390000/dx);
m       = 200000/(90*dx)
for i in 1:sizeC   
    # profile A
    xCA[i] = profile_size - deltaXC - round(m*(i - 1));
    yCA[i] = profile_size + (i - 1);
    # profile B
    xCB[i] = profile_size + deltaXC + round(m*(i - 1));
    yCB[i] = profile_size + (i - 1);
    # profile C
    xCC[i] = profile_size - deltaXC - round(m*(i - 1));
    yCC[i] = profile_size - (i - 1);
    # profile D
    xCD[i] = profile_size + deltaXC + round(m*(i - 1));
    yCD[i] = profile_size - (i - 1);
end

# === Halbrane ===
deltaXH = Int(150000/dx);
for i in 1:sizeH   
    # profile A
    xHA[i] = profile_size - deltaXH;
    yHA[i] = profile_size + (i - 1);
    # profile B
    xHB[i] = profile_size + deltaXH;
    yHB[i] = profile_size + (i - 1);
    # profile C
    xHC[i] = profile_size - deltaXH;
    yHC[i] = profile_size - (i - 1);
    # profile D
    xHD[i] = profile_size + deltaXH;
    yHD[i] = profile_size - (i - 1);
end

In [67]:
(xCA[sizeC]-profile_size)*dx

-590000.0

In [68]:
# === CXaprona ===
Caprona_A_H    = zeros(sizeC,tsize);
Caprona_A_Mask = zeros(sizeC,tsize);
Caprona_A_S    = zeros(sizeC,tsize);
Caprona_A_VX   = zeros(sizeC,tsize);
Caprona_A_VY   = zeros(sizeC,tsize);

Caprona_B_H    = zeros(sizeC,tsize);
Caprona_B_Mask = zeros(sizeC,tsize);
Caprona_B_S    = zeros(sizeC,tsize);
Caprona_B_VX   = zeros(sizeC,tsize);
Caprona_B_VY   = zeros(sizeC,tsize);

Caprona_C_H    = zeros(sizeC,tsize);
Caprona_C_Mask = zeros(sizeC,tsize);
Caprona_C_S    = zeros(sizeC,tsize);
Caprona_C_VX   = zeros(sizeC,tsize);
Caprona_C_VY   = zeros(sizeC,tsize);

Caprona_D_H    = zeros(sizeC,tsize);
Caprona_D_Mask = zeros(sizeC,tsize);
Caprona_D_S    = zeros(sizeC,tsize);
Caprona_D_VX   = zeros(sizeC,tsize);
Caprona_D_VY   = zeros(sizeC,tsize);

# === Halbrane ===
Halbrane_A_H    = zeros(sizeH,tsize);
Halbrane_A_Mask = zeros(sizeH,tsize);
Halbrane_A_S    = zeros(sizeH,tsize);
Halbrane_A_VX   = zeros(sizeH,tsize);
Halbrane_A_VY   = zeros(sizeH,tsize);

Halbrane_B_H    = zeros(sizeH,tsize);
Halbrane_B_Mask = zeros(sizeH,tsize);
Halbrane_B_S    = zeros(sizeH,tsize);
Halbrane_B_VX   = zeros(sizeH,tsize);
Halbrane_B_VY   = zeros(sizeH,tsize);

Halbrane_C_H    = zeros(sizeH,tsize);
Halbrane_C_Mask = zeros(sizeH,tsize);
Halbrane_C_S    = zeros(sizeH,tsize);
Halbrane_C_VX   = zeros(sizeH,tsize);
Halbrane_C_VY   = zeros(sizeH,tsize);

Halbrane_D_H    = zeros(sizeH,tsize);
Halbrane_D_Mask = zeros(sizeH,tsize);
Halbrane_D_S    = zeros(sizeH,tsize);
Halbrane_D_VX   = zeros(sizeH,tsize);
Halbrane_D_VY   = zeros(sizeH,tsize);

# 1D fields (time)
xcfCapA        = zeros(tsize);
ycfCapA        = zeros(tsize);
xvelmeancfCapA = zeros(tsize);
yvelmeancfCapA = zeros(tsize);
lithkcfCapA    = zeros(tsize);

xcfCapB        = zeros(tsize);
ycfCapB        = zeros(tsize);
xvelmeancfCapB = zeros(tsize);
yvelmeancfCapB = zeros(tsize);
lithkcfCapB    = zeros(tsize);

xcfCapC        = zeros(tsize);
ycfCapC        = zeros(tsize);
xvelmeancfCapC = zeros(tsize);
yvelmeancfCapC = zeros(tsize);
lithkcfCapC    = zeros(tsize);

xcfCapD        = zeros(tsize);
ycfCapD        = zeros(tsize);
xvelmeancfCapD = zeros(tsize);
yvelmeancfCapD = zeros(tsize);
lithkcfCapD    = zeros(tsize);

xcfHalA        = zeros(tsize);
ycfHalA        = zeros(tsize);
xvelmeancfHalA = zeros(tsize);
yvelmeancfHalA = zeros(tsize);
lithkcfHalA    = zeros(tsize);

xcfHalB        = zeros(tsize);
ycfHalB        = zeros(tsize);
xvelmeancfHalB = zeros(tsize);
yvelmeancfHalB = zeros(tsize);
lithkcfHalB    = zeros(tsize);

xcfHalC        = zeros(tsize);
ycfHalC        = zeros(tsize);
xvelmeancfHalC = zeros(tsize);
yvelmeancfHalC = zeros(tsize);
lithkcfHalC    = zeros(tsize);

xcfHalD        = zeros(tsize);
ycfHalD        = zeros(tsize);
xvelmeancfHalD = zeros(tsize);
yvelmeancfHalD = zeros(tsize);
lithkcfHalD    = zeros(tsize);

In [69]:
# === Caprona ===
for i in 1:tsize
    # count to check the ice front
    countA = 0;
    countB = 0;
    countC = 0;
    countD = 0;

    for j in 1:sizeC
        # == profile A ===
        Caprona_A_H[j,i]    = H_ice[Int(xCA[j]),Int(yCA[j]),i]
        Caprona_A_S[j,i]    = dx*((xCA[j]-xCA[1])^2 + (yCA[j]-yCA[1])^2)^0.5
        Caprona_A_VX[j,i]   = ux[Int(xCA[j]),Int(yCA[j]),i]
        Caprona_A_VY[j,i]   = vy[Int(xCA[j]),Int(yCA[j]),i]
        Caprona_A_Mask[j,i] = mask[Int(xCA[j]),Int(yCA[j]),i]

        if((countA == 0 && Caprona_A_H[j,i] < -100.0) || (countA == 0 && j==sizeC))
            countA             = 1;
            xcfCapA[i]         = dx*(xCA[j-1]-xCA[1]);
            ycfCapA[i]         = dx*(yCA[j-1]-yCA[1]);
            xvelmeancfCapA[i]  = ux_ac[Int(xCA[j-1]),Int(yCA[j-1]),i]; 
            yvelmeancfCapA[i]  = vy_ac[Int(xCA[j-1]),Int(yCA[j-1]-1),i]; 
            #xvelmeancfCapA[i]  = Caprona_A_VX[j-1,i];
            #yvelmeancfCapA[i]  = Caprona_A_VY[j-1,i];
            lithkcfCapA[i]     = H_frnt[Int(xCA[j-1]),Int(yCA[j-1]),i];#Caprona_A_H[j-1,i];
        end 

        # == profile B ===
        Caprona_B_H[j,i]    = H_ice[Int(xCB[j]),Int(yCB[j]),i]
        Caprona_B_S[j,i]    = dx*((xCB[j]-xCB[1])^2 + (yCB[j]-yCB[1])^2)^0.5
        Caprona_B_VX[j,i]   = ux[Int(xCB[j]),Int(yCB[j]),i]
        Caprona_B_VY[j,i]   = vy[Int(xCB[j]),Int(yCB[j]),i]
        Caprona_B_Mask[j,i] = mask[Int(xCB[j]),Int(yCB[j]),i]

        if((countB == 0 && Caprona_B_H[j,i] < -100.0) || (countB == 0 && j==sizeC))
            countB             = 1;
            xcfCapB[i]         = dx*(xCB[j-1]-xCB[1]);
            ycfCapB[i]         = dx*(yCB[j-1]-yCB[1]);
            xvelmeancfCapB[i]  = ux_ac[Int(xCB[j-1]-1),Int(yCB[j-1]),i]; 
            yvelmeancfCapB[i]  = vy_ac[Int(xCB[j-1]),Int(yCB[j-1]-1),i]; 
            #xvelmeancfCapB[i]  = Caprona_B_VX[j-1,i];
            #yvelmeancfCapB[i]  = Caprona_B_VY[j-1,i];
            lithkcfCapB[i]     = H_frnt[Int(xCB[j-1]),Int(yCB[j-1]),i];#Caprona_B_H[j-1,i];
        end 

        # == profile C ===
        Caprona_C_H[j,i]    = H_ice[Int(xCC[j]),Int(yCC[j]),i]
        Caprona_C_S[j,i]    = dx*((xCC[j]-xCC[1])^2 + (yCC[j]-yCC[1])^2)^0.5
        Caprona_C_VX[j,i]   = ux[Int(xCC[j]),Int(yCC[j]),i]
        Caprona_C_VY[j,i]   = vy[Int(xCC[j]),Int(yCC[j]),i]
        Caprona_C_Mask[j,i] = mask[Int(xCC[j]),Int(yCC[j]),i]

        if((countC == 0 && Caprona_C_H[j,i] < -100.0) || (countC == 0 && j==sizeC))
            countC             = 1;
            xcfCapC[i]         = dx*(xCC[j-1]-xCC[1]);
            ycfCapC[i]         = dx*(yCC[j-1]-yCC[1]);
            xvelmeancfCapC[i]  = ux_ac[Int(xCC[j-1]-1),Int(yCC[j-1]),i]; 
            yvelmeancfCapC[i]  = vy_ac[Int(xCC[j-1]),Int(yCC[j-1]),i]; 
            #xvelmeancfCapC[i]  = Caprona_C_VX[j-1,i];
            #yvelmeancfCapC[i]  = Caprona_C_VY[j-1,i];
            lithkcfCapC[i]     = H_frnt[Int(xCC[j-1]),Int(yCC[j-1]),i];#Caprona_C_H[j-1,i];
        end 

        # == profile D ===
        Caprona_D_H[j,i]    = H_ice[Int(xCD[j]),Int(yCD[j]),i]
        Caprona_D_S[j,i]    = dx*((xCD[j]-xCD[1])^2 + (yCD[j]-yCD[1])^2)^0.5
        Caprona_D_VX[j,i]   = ux[Int(xCD[j]),Int(yCD[j]),i]
        Caprona_D_VY[j,i]   = vy[Int(xCD[j]),Int(yCD[j]),i]
        Caprona_D_Mask[j,i] = mask[Int(xCD[j]),Int(yCD[j]),i]

        if((countD == 0 && Caprona_D_H[j,i] < -100.0) || (countD == 0 && j==sizeC))
            countD             = 1;
            xcfCapD[i]         = dx*(xCD[j-1]-xCD[1]);
            ycfCapD[i]         = dx*(yCD[j-1]-yCD[1]);
            xvelmeancfCapD[i]  = ux_ac[Int(xCD[j-1]),Int(yCD[j-1]),i]; 
            yvelmeancfCapD[i]  = vy_ac[Int(xCD[j-1]),Int(yCD[j-1]),i]; 
            #xvelmeancfCapD[i]  = Caprona_D_VX[j-1,i];
            #yvelmeancfCapD[i]  = Caprona_D_VY[j-1,i];
            lithkcfCapD[i]     = H_frnt[Int(xCD[j-1]),Int(yCD[j-1]),i];#Caprona_D_H[j-1,i];
        end 
    end
end

In [70]:
# === Halbrane ===
for i in 1:tsize
    # count to check the ice front
    countA = 0;
    countB = 0;
    countC = 0;
    countD = 0;

    for j in 1:sizeH
        # == profile A ===
        Halbrane_A_H[j,i]    = H_ice[Int(xHA[j]),Int(yHA[j]),i]
        Halbrane_A_S[j,i]    = dx*((xHA[j]-xHA[1])^2 + (yHA[j]-yHA[1])^2)^0.5
        Halbrane_A_VX[j,i]   = ux[Int(xHA[j]),Int(yHA[j]),i]
        Halbrane_A_VY[j,i]   = vy[Int(xHA[j]),Int(yHA[j]),i]
        Halbrane_A_Mask[j,i] = mask[Int(xHA[j]),Int(yHA[j]),i]

        if((countA == 0 && Halbrane_A_H[j,i] < -100.0) || (countA == 0 && j==sizeH))
            countA             = 1;
            xcfHalA[i]         = dx*(xHA[j-1]-xHA[1]);
            ycfHalA[i]         = dx*(yHA[j-1]-yHA[1]);
            xvelmeancfHalA[i]  = 0.5*(ux_ac[Int(xHA[j-1]-1),Int(yHA[j-1]),i] + ux_ac[Int(xHA[j-1]),Int(yHA[j-1]),i]);
            yvelmeancfHalA[i]  = vy_ac[Int(xHA[j-1]),Int(yHA[j-1]-1),i]; 
            #xvelmeancfHalA[i]  = Halbrane_A_VX[j-1,i];
            #yvelmeancfHalA[i]  = Halbrane_A_VY[j-1,i];
            lithkcfHalA[i]     = H_frnt[Int(xHA[j-1]),Int(yHA[j-1]),i];#Halbrane_A_H[j-1,i];
        end 

        # == profile B ===
        Halbrane_B_H[j,i]    = H_ice[Int(xHB[j]),Int(yHB[j]),i]
        Halbrane_B_S[j,i]    = dx*((xHB[j]-xHB[1])^2 + (yHB[j]-yHB[1])^2)^0.5
        Halbrane_B_VX[j,i]   = ux[Int(xHB[j]),Int(yHB[j]),i]
        Halbrane_B_VY[j,i]   = vy[Int(xHB[j]),Int(yHB[j]),i]
        Halbrane_B_Mask[j,i] = mask[Int(xHB[j]),Int(yHB[j]),i]

        if((countB == 0 && Halbrane_B_H[j,i] < -100.0) || (countB == 0 && j==sizeH))
            countB             = 1;
            xcfHalB[i]         = dx*(xHB[j-1]-xHB[1]);
            ycfHalB[i]         = dx*(yHB[j-1]-yHB[1]);
            xvelmeancfHalB[i]  = 0.5*(ux_ac[Int(xHB[j-1]-1),Int(yHB[j-1]),i] + ux_ac[Int(xHB[j-1]),Int(yHB[j-1]),i]);  
            yvelmeancfHalB[i]  = vy_ac[Int(xHB[j-1]),Int(yHB[j-1]-1),i]; 
            #xvelmeancfHalB[i]  = Halbrane_B_VX[j-1,i];
            #yvelmeancfHalB[i]  = Halbrane_B_VY[j-1,i];
            lithkcfHalB[i]     = H_frnt[Int(xHB[j-1]),Int(yHB[j-1]),i];#Halbrane_B_H[j-1,i];
        end 

        # == profile C ===
        Halbrane_C_H[j,i]    = H_ice[Int(xHC[j]),Int(yHC[j]),i]
        Halbrane_C_S[j,i]    = dx*((xHC[j]-xHC[1])^2 + (yHC[j]-yHC[1])^2)^0.5
        Halbrane_C_VX[j,i]   = ux[Int(xHC[j]),Int(yHC[j]),i]
        Halbrane_C_VY[j,i]   = vy[Int(xHC[j]),Int(yHC[j]),i]
        Halbrane_C_Mask[j,i] = mask[Int(xHC[j]),Int(yHC[j]),i]

        if((countC == 0 && Halbrane_C_H[j,i] < -100.0) || (countC == 0 && j==sizeH))
            countC             = 1;
            xcfHalC[i]         = dx*(xHC[j-1]-xHC[1]);
            ycfHalC[i]         = dx*(yHC[j-1]-yHC[1]);
            xvelmeancfHalC[i]  = 0.5*(ux_ac[Int(xHC[j-1]-1),Int(yHC[j-1]),i] + ux_ac[Int(xHC[j-1]),Int(yHC[j-1]),i]);  
            yvelmeancfHalC[i]  = vy_ac[Int(xHC[j-1]),Int(yHC[j-1]),i]; 
            #xvelmeancfHalC[i]  = Halbrane_C_VX[j-1,i];
            #yvelmeancfHalC[i]  = Halbrane_C_VY[j-1,i];
            lithkcfHalC[i]     = H_frnt[Int(xHC[j-1]),Int(yHC[j-1]),i];#Halbrane_C_H[j-1,i];
        end 

        # == profile D ===
        Halbrane_D_H[j,i]    = H_ice[Int(xHD[j]),Int(yHD[j]),i]
        Halbrane_D_S[j,i]    = dx*((xHD[j]-xHD[1])^2 + (yHD[j]-yHD[1])^2)^0.5
        Halbrane_D_VX[j,i]   = ux[Int(xHD[j]),Int(yHD[j]),i]
        Halbrane_D_VY[j,i]   = vy[Int(xHD[j]),Int(yHD[j]),i]
        Halbrane_D_Mask[j,i] = mask[Int(xHD[j]),Int(yHD[j]),i]

        if((countD == 0 && Halbrane_D_H[j,i] < -100.0) || (countD == 0 && j==sizeH))
            countD             = 1;
            xcfHalD[i]         = dx*(xHD[j-1]-xHD[1]);
            ycfHalD[i]         = dx*(yHD[j-1]-yHD[1]);
            xvelmeancfHalD[i]  = 0.5*(ux_ac[Int(xHD[j-1]-1),Int(yHD[j-1]),i] + ux_ac[Int(xHD[j-1]),Int(yHD[j-1]),i]);  
            yvelmeancfHalD[i]  = vy_ac[Int(xHD[j-1]),Int(yHD[j-1]),i]; 
            #xvelmeancfHalD[i]  = Halbrane_D_VX[j-1,i];
            #yvelmeancfHalD[i]  = Halbrane_D_VY[j-1,i];
            lithkcfHalD[i]     = H_frnt[Int(xHD[j-1]),Int(yHD[j-1]),i];#Halbrane_D_H[j-1,i];
        end 
    end
end

#### Caprona

In [71]:
# Output file path
ds_out = "profiles/Caprona2D.nc"
filepath = joinpath(ds_path, exp, ds_out)

# Ensure the file doesn't exist before writing
if isfile(filepath)
    rm(filepath, force=true)
    sleep(0.5)  # Short pause to ensure deletion
end

ds = NCDataset(filepath, "c")

# Define dimensions
defDim(ds, "Caprona A", sizeC)
defDim(ds, "Caprona B", sizeC)
defDim(ds, "Caprona C", sizeC)
defDim(ds, "Caprona D", sizeC)
defDim(ds, "Time", tsize)

# Define the coordinate variables
v_xc_A = defVar(ds, "Caprona A", Float32, ("Caprona A",))
v_xc_B = defVar(ds, "Caprona B", Float32, ("Caprona B",))
v_xc_C = defVar(ds, "Caprona C", Float32, ("Caprona C",))
v_xc_D = defVar(ds, "Caprona D", Float32, ("Caprona D",))
v_time = defVar(ds, "Time", Float32, ("Time",))

# Assign depth values as the coordinate values
v_xc_A[:] = 1:sizeC
v_xc_B[:] = 1:sizeC
v_xc_C[:] = 1:sizeC
v_xc_D[:] = 1:sizeC
v_time[:] = 0:tsize-1

# Define variables for Profile A
defVar(ds, "Caprona_A_H", Float32, ("Caprona A", "Time"), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "land_ice_thickness_along_profile_A",
))[:, :] = Caprona_A_H

defVar(ds, "Caprona_A_S", Float32, ("Caprona A", "Time"), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "distance_along_profile_A",
))[:, :] = Caprona_A_S

defVar(ds, "Caprona_A_VX", Float32, ("Caprona A", "Time"), attrib = OrderedDict(
    "units" => "m a-1",
    "standard_name" => "land_ice_vertical_mean_x_velocity_along_profile_A",
))[:, :] = Caprona_A_VX

defVar(ds, "Caprona_A_VY", Float32, ("Caprona A", "Time"), attrib = OrderedDict(
    "units" => "m a-1",
    "standard_name" => "land_ice_vertical_mean_y_velocity_along_profile_A",
))[:, :] = Caprona_A_VY

defVar(ds, "Caprona_A_Mask", Int32, ("Caprona A", "Time"), attrib = OrderedDict(
    "flag_values" => "1, 2, 3",
    "flag_meanings" => "1=grounded ice, 2=floating ice, 3=open ocean",
))[:, :] = Caprona_A_Mask

# Define variables for Profile B
defVar(ds, "Caprona_B_H", Float32, ("Caprona B", "Time"), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "land_ice_thickness_along_profile_B",
))[:, :] = Caprona_B_H

defVar(ds, "Caprona_B_S", Float32, ("Caprona B", "Time"), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "distance_along_profile_B",
))[:, :] = Caprona_B_S

defVar(ds, "Caprona_B_VX", Float32, ("Caprona B", "Time"), attrib = OrderedDict(
    "units" => "m a-1",
    "standard_name" => "land_ice_vertical_mean_x_velocity_along_profile_B",
))[:, :] = Caprona_B_VX

defVar(ds, "Caprona_B_VY", Float32, ("Caprona B", "Time"), attrib = OrderedDict(
    "units" => "m a-1",
    "standard_name" => "land_ice_vertical_mean_y_velocity_along_profile_B",
))[:, :] = Caprona_B_VY

defVar(ds, "Caprona_B_Mask", Int32, ("Caprona B", "Time"), attrib = OrderedDict(
    "flag_values" => "1, 2, 3",
    "flag_meanings" => "1=grounded ice, 2=floating ice, 3=open ocean",
))[:, :] = Caprona_B_Mask

# Define variables for Profile C
defVar(ds, "Caprona_C_H", Float32, ("Caprona C", "Time"), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "land_ice_thickness_along_profile_C",
))[:, :] = Caprona_C_H

defVar(ds, "Caprona_C_S", Float32, ("Caprona C", "Time"), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "distance_along_profile_C",
))[:, :] = Caprona_C_S

defVar(ds, "Caprona_C_VX", Float32, ("Caprona C", "Time"), attrib = OrderedDict(
    "units" => "m a-1",
    "standard_name" => "land_ice_vertical_mean_x_velocity_along_profile_C",
))[:, :] = Caprona_C_VX

defVar(ds, "Caprona_C_VY", Float32, ("Caprona C", "Time"), attrib = OrderedDict(
    "units" => "m a-1",
    "standard_name" => "land_ice_vertical_mean_y_velocity_along_profile_C",
))[:, :] = Caprona_C_VY

defVar(ds, "Caprona_C_Mask", Int32, ("Caprona C", "Time"), attrib = OrderedDict(
    "flag_values" => "1, 2, 3",
    "flag_meanings" => "1=grounded ice, 2=floating ice, 3=open ocean",
))[:, :] = Caprona_C_Mask

# Define variables for Profile D
defVar(ds, "Caprona_D_H", Float32, ("Caprona D", "Time"), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "land_ice_thickness_along_profile_D",
))[:, :] = Caprona_D_H

defVar(ds, "Caprona_D_S", Float32, ("Caprona D", "Time"), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "distance_along_profile_D",
))[:, :] = Caprona_D_S

defVar(ds, "Caprona_D_VX", Float32, ("Caprona D", "Time"), attrib = OrderedDict(
    "units" => "m a-1",
    "standard_name" => "land_ice_vertical_mean_x_velocity_along_profile_D",
))[:, :] = Caprona_D_VX

defVar(ds, "Caprona_D_VY", Float32, ("Caprona D", "Time"), attrib = OrderedDict(
    "units" => "m a-1",
    "standard_name" => "land_ice_vertical_mean_y_velocity_along_profile_D",
))[:, :] = Caprona_D_VY

defVar(ds, "Caprona_D_Mask", Int32, ("Caprona D", "Time"), attrib = OrderedDict(
    "flag_values" => "1, 2, 3",
    "flag_meanings" => "1=grounded ice, 2=floating ice, 3=open ocean",
))[:, :] = Caprona_D_Mask

close(ds)


closed Dataset

In [72]:
# Output file path
ds_out = "profiles/Caprona1D.nc"
filepath = joinpath(ds_path, exp, ds_out)

# Ensure the file doesn't exist before writing
if isfile(filepath)
    rm(filepath, force=true)
    sleep(0.5)  # Short pause to ensure deletion
end

# Create a new NetCDF dataset
ds = NCDataset(filepath, "c")

# Define dimensions
defDim(ds, "Time1", tsize)

# Define the coordinate variable
v_time = defVar(ds, "Time1", Float64, ("Time1",))
v_time[:] = 0:tsize-1

# Define variables for Caprona Profile A
defVar(ds, "lithkcfCapA", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "land_ice_thickness_at_calving_front_on_profile_CapA",
))[:] = lithkcfCapA

defVar(ds, "xcfCapA", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "x_calving_front_on_profile_CapA",
))[:] = xcfCapA

defVar(ds, "ycfCapA", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "y_calving_front_on_profile_CapA",
))[:] = ycfCapA

defVar(ds, "xvelmeancfCapA", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m yr-1",
    "standard_name" => "land_ice_vertical_mean_x_velocity_at_calving_front_on_profile_CapA",
))[:] = xvelmeancfCapA

defVar(ds, "yvelmeancfCapA", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m yr-1",
    "standard_name" => "land_ice_vertical_mean_y_velocity_at_calving_front_on_profile_CapA",
))[:] = yvelmeancfCapA

# Define variables for Caprona Profile B
defVar(ds, "lithkcfCapB", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "land_ice_thickness_at_calving_front_on_profile_CapB",
))[:] = lithkcfCapB

defVar(ds, "xcfCapB", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "x_calving_front_on_profile_CapB",
))[:] = xcfCapB

defVar(ds, "ycfCapB", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "y_calving_front_on_profile_CapB",
))[:] = ycfCapB

defVar(ds, "xvelmeancfCapB", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m yr-1",
    "standard_name" => "land_ice_vertical_mean_x_velocity_at_calving_front_on_profile_CapB",
))[:] = xvelmeancfCapB

defVar(ds, "yvelmeancfCapB", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m yr-1",
    "standard_name" => "land_ice_vertical_mean_y_velocity_at_calving_front_on_profile_CapB",
))[:] = yvelmeancfCapB

# Define variables for Caprona Profile C
defVar(ds, "lithkcfCapC", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "land_ice_thickness_at_calving_front_on_profile_CapC",
))[:] = lithkcfCapC

defVar(ds, "xcfCapC", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "x_calving_front_on_profile_CapC",
))[:] = xcfCapC

defVar(ds, "ycfCapC", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "y_calving_front_on_profile_CapC",
))[:] = ycfCapC

defVar(ds, "xvelmeancfCapC", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m yr-1",
    "standard_name" => "land_ice_vertical_mean_x_velocity_at_calving_front_on_profile_CapC",
))[:] = xvelmeancfCapC

defVar(ds, "yvelmeancfCapC", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m yr-1",
    "standard_name" => "land_ice_vertical_mean_y_velocity_at_calving_front_on_profile_CapC",
))[:] = yvelmeancfCapC

# Define variables for Caprona Profile D
defVar(ds, "lithkcfCapD", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "land_ice_thickness_at_calving_front_on_profile_CapD",
))[:] = lithkcfCapD

defVar(ds, "xcfCapD", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "x_calving_front_on_profile_CapD",
))[:] = xcfCapD

defVar(ds, "ycfCapD", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "y_calving_front_on_profile_CapD",
))[:] = ycfCapD

defVar(ds, "xvelmeancfCapD", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m yr-1",
    "standard_name" => "land_ice_vertical_mean_x_velocity_at_calving_front_on_profile_CapD",
))[:] = xvelmeancfCapD

defVar(ds, "yvelmeancfCapD", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m yr-1",
    "standard_name" => "land_ice_vertical_mean_y_velocity_at_calving_front_on_profile_CapD",
))[:] = yvelmeancfCapD

# Close the dataset
close(ds)


closed Dataset

### Halbrane

In [73]:
# Output file path
ds_out = "profiles/Halbrane2D.nc"
filepath = joinpath(ds_path, exp, ds_out)

# Ensure the file doesn't exist before writing
if isfile(filepath)
    rm(filepath, force=true)
    sleep(0.5)  # Short pause to ensure deletion
end

ds = NCDataset(filepath, "c")

# Define dimensions
defDim(ds, "Halbrane A", sizeH)
defDim(ds, "Halbrane B", sizeH)
defDim(ds, "Halbrane C", sizeH)
defDim(ds, "Halbrane D", sizeH)
defDim(ds, "Time", tsize)

# Define the coordinate variables
v_xc_A = defVar(ds, "Halbrane A", Float32, ("Halbrane A",))
v_xc_B = defVar(ds, "Halbrane B", Float32, ("Halbrane B",))
v_xc_C = defVar(ds, "Halbrane C", Float32, ("Halbrane C",))
v_xc_D = defVar(ds, "Halbrane D", Float32, ("Halbrane D",))
v_time = defVar(ds, "Time", Float32, ("Time",))

# Assign depth values as the coordinate values
v_xc_A[:] = 1:sizeH
v_xc_B[:] = 1:sizeH
v_xc_C[:] = 1:sizeH
v_xc_D[:] = 1:sizeH
v_time[:] = 0:tsize-1

# Define variables for Profile A
defVar(ds, "Halbrane_A_H", Float32, ("Halbrane A", "Time"), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "land_ice_thickness_along_profile_A",
))[:, :] = Halbrane_A_H

defVar(ds, "Halbrane_A_S", Float32, ("Halbrane A", "Time"), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "distance_along_profile_A",
))[:, :] = Halbrane_A_S

defVar(ds, "Halbrane_A_VX", Float32, ("Halbrane A", "Time"), attrib = OrderedDict(
    "units" => "m a-1",
    "standard_name" => "land_ice_vertical_mean_x_velocity_along_profile_A",
))[:, :] = Halbrane_A_VX

defVar(ds, "Halbrane_A_VY", Float32, ("Halbrane A", "Time"), attrib = OrderedDict(
    "units" => "m a-1",
    "standard_name" => "land_ice_vertical_mean_y_velocity_along_profile_A",
))[:, :] = Halbrane_A_VY

defVar(ds, "Halbrane_A_Mask", Int32, ("Halbrane A", "Time"), attrib = OrderedDict(
    "flag_values" => "1, 2, 3",
    "flag_meanings" => "1=grounded ice, 2=floating ice, 3=open ocean",
))[:, :] = Halbrane_A_Mask

# Define variables for Profile B
defVar(ds, "Halbrane_B_H", Float32, ("Halbrane B", "Time"), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "land_ice_thickness_along_profile_B",
))[:, :] = Halbrane_B_H

defVar(ds, "Halbrane_B_S", Float32, ("Halbrane B", "Time"), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "distance_along_profile_B",
))[:, :] = Halbrane_B_S

defVar(ds, "Halbrane_B_VX", Float32, ("Halbrane B", "Time"), attrib = OrderedDict(
    "units" => "m a-1",
    "standard_name" => "land_ice_vertical_mean_x_velocity_along_profile_B",
))[:, :] = Halbrane_B_VX

defVar(ds, "Halbrane_B_VY", Float32, ("Halbrane B", "Time"), attrib = OrderedDict(
    "units" => "m a-1",
    "standard_name" => "land_ice_vertical_mean_y_velocity_along_profile_B",
))[:, :] = Halbrane_B_VY

defVar(ds, "Halbrane_B_Mask", Int32, ("Halbrane B", "Time"), attrib = OrderedDict(
    "flag_values" => "1, 2, 3",
    "flag_meanings" => "1=grounded ice, 2=floating ice, 3=open ocean",
))[:, :] = Halbrane_B_Mask

# Define variables for Profile C
defVar(ds, "Halbrane_C_H", Float32, ("Halbrane C", "Time"), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "land_ice_thickness_along_profile_C",
))[:, :] = Halbrane_C_H

defVar(ds, "Halbrane_C_S", Float32, ("Halbrane C", "Time"), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "distance_along_profile_C",
))[:, :] = Halbrane_C_S

defVar(ds, "Halbrane_C_VX", Float32, ("Halbrane C", "Time"), attrib = OrderedDict(
    "units" => "m a-1",
    "standard_name" => "land_ice_vertical_mean_x_velocity_along_profile_C",
))[:, :] = Halbrane_C_VX

defVar(ds, "Halbrane_C_VY", Float32, ("Halbrane C", "Time"), attrib = OrderedDict(
    "units" => "m a-1",
    "standard_name" => "land_ice_vertical_mean_y_velocity_along_profile_C",
))[:, :] = Halbrane_C_VY

defVar(ds, "Halbrane_C_Mask", Int32, ("Halbrane C", "Time"), attrib = OrderedDict(
    "flag_values" => "1, 2, 3",
    "flag_meanings" => "1=grounded ice, 2=floating ice, 3=open ocean",
))[:, :] = Halbrane_C_Mask

# Define variables for Profile D
defVar(ds, "Halbrane_D_H", Float32, ("Halbrane D", "Time"), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "land_ice_thickness_along_profile_D",
))[:, :] = Halbrane_D_H

defVar(ds, "Halbrane_D_S", Float32, ("Halbrane D", "Time"), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "distance_along_profile_D",
))[:, :] = Halbrane_D_S

defVar(ds, "Halbrane_D_VX", Float32, ("Halbrane D", "Time"), attrib = OrderedDict(
    "units" => "m a-1",
    "standard_name" => "land_ice_vertical_mean_x_velocity_along_profile_D",
))[:, :] = Halbrane_D_VX

defVar(ds, "Halbrane_D_VY", Float32, ("Halbrane D", "Time"), attrib = OrderedDict(
    "units" => "m a-1",
    "standard_name" => "land_ice_vertical_mean_y_velocity_along_profile_D",
))[:, :] = Halbrane_D_VY

defVar(ds, "Halbrane_D_Mask", Int32, ("Halbrane D", "Time"), attrib = OrderedDict(
    "flag_values" => "1, 2, 3",
    "flag_meanings" => "1=grounded ice, 2=floating ice, 3=open ocean",
))[:, :] = Halbrane_D_Mask

close(ds)


closed Dataset

In [74]:
# Output file path
ds_out = "profiles/Halbrane1D.nc"
filepath = joinpath(ds_path, exp, ds_out)

# Ensure the file doesn't exist before writing
if isfile(filepath)
    rm(filepath, force=true)
    sleep(0.5)  # Short pause to ensure deletion
end

# Create a new NetCDF dataset
ds = NCDataset(filepath, "c")

# Define dimensions
defDim(ds, "Time1", tsize)

# Define the coordinate variable
v_time = defVar(ds, "Time1", Float64, ("Time1",))
v_time[:] = 0:tsize-1

# Define variables for Halbrane Profile A
defVar(ds, "lithkcfHalA", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "land_ice_thickness_at_calving_front_on_profile_HalA",
))[:] = lithkcfHalA

defVar(ds, "xcfHalA", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "x_calving_front_on_profile_HalA",
))[:] = xcfHalA

defVar(ds, "ycfHalA", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "y_calving_front_on_profile_HalA",
))[:] = ycfHalA

defVar(ds, "xvelmeancfHalA", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m yr-1",
    "standard_name" => "land_ice_vertical_mean_x_velocity_at_calving_front_on_profile_HalA",
))[:] = xvelmeancfHalA

defVar(ds, "yvelmeancfHalA", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m yr-1",
    "standard_name" => "land_ice_vertical_mean_y_velocity_at_calving_front_on_profile_HalA",
))[:] = yvelmeancfHalA

# Define variables for Halbrane Profile B
defVar(ds, "lithkcfHalB", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "land_ice_thickness_at_calving_front_on_profile_HalB",
))[:] = lithkcfHalB

defVar(ds, "xcfHalB", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "x_calving_front_on_profile_HalB",
))[:] = xcfHalB

defVar(ds, "ycfHalB", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "y_calving_front_on_profile_HalB",
))[:] = ycfHalB

defVar(ds, "xvelmeancfHalB", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m yr-1",
    "standard_name" => "land_ice_vertical_mean_x_velocity_at_calving_front_on_profile_HalB",
))[:] = xvelmeancfHalB

defVar(ds, "yvelmeancfHalB", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m yr-1",
    "standard_name" => "land_ice_vertical_mean_y_velocity_at_calving_front_on_profile_HalB",
))[:] = yvelmeancfHalB

# Define variables for Halbrane Profile C
defVar(ds, "lithkcfHalC", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "land_ice_thickness_at_calving_front_on_profile_HalC",
))[:] = lithkcfHalC

defVar(ds, "xcfHalC", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "x_calving_front_on_profile_HalC",
))[:] = xcfHalC

defVar(ds, "ycfHalC", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "y_calving_front_on_profile_HalC",
))[:] = ycfHalC

defVar(ds, "xvelmeancfHalC", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m yr-1",
    "standard_name" => "land_ice_vertical_mean_x_velocity_at_calving_front_on_profile_HalC",
))[:] = xvelmeancfHalC

defVar(ds, "yvelmeancfHalC", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m yr-1",
    "standard_name" => "land_ice_vertical_mean_y_velocity_at_calving_front_on_profile_HalC",
))[:] = yvelmeancfHalC

# Define variables for Halbrane Profile D
defVar(ds, "lithkcfHalD", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "land_ice_thickness_at_calving_front_on_profile_HalD",
))[:] = lithkcfHalD

defVar(ds, "xcfHalD", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "x_calving_front_on_profile_HalD",
))[:] = xcfHalD

defVar(ds, "ycfHalD", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m",
    "standard_name" => "y_calving_front_on_profile_HalD",
))[:] = ycfHalD

defVar(ds, "xvelmeancfHalD", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m yr-1",
    "standard_name" => "land_ice_vertical_mean_x_velocity_at_calving_front_on_profile_HalD",
))[:] = xvelmeancfHalD

defVar(ds, "yvelmeancfHalD", Float64, ("Time1",), attrib = OrderedDict(
    "units" => "m yr-1",
    "standard_name" => "land_ice_vertical_mean_y_velocity_at_calving_front_on_profile_HalD",
))[:] = yvelmeancfHalD

# Close the dataset
close(ds)


closed Dataset